# Import Library and Load Data

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot
import os
import time
import cv2

from math import sin, cos, pi
from pandas import DataFrame
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# loading data

Train_Dir = '../input/facial-keypoints-detection/training.zip'
Test_Dir = '../input/facial-keypoints-detection/test.zip'
lookid_dir = '../input/facial-keypoints-detection/IdLookupTable.csv'
lookid_data = pd.read_csv(lookid_dir)

def load(test = False, cols = None):
    """
    Loads the dataset.
    Returns a tuple of X and y, if `test` was set to `True` y contains `None`.    
    """
    
    fname = Test_Dir if test else Train_Dir
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep = ' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]
    print("There are missing points:")
    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None
        
    return X.reshape(df.shape[0],96,96), y

def plot_sample(x, y):
    """
    Plots a single sample image with keypoints on top.   
    """
    pyplot.imshow(x, cmap='gray')
    pyplot.scatter(y[0::2] * 48 + 48, y[1::2] * 48 + 48, marker='x', s=10)

In [ ]:
x_train, y_train = load()
x_train = np.expand_dims(x_train, -1)
plot_sample(x_train[10],y_train[10])

In [ ]:
x_test, y_test = load(True)
x_test = np.expand_dims(x_test, -1)

In [ ]:
x_test.shape

In [ ]:
x_train.shape

# Data Augmentation

In [ ]:
rotation_angles = [12]  # Rotation Degree
pixel_shifts = [12] # Pixel Moved
x_train_original = x_train
y_train_original = y_train

# Bool Variable to trigger augmentation
rotation_augmentation = True # Only use rotation_augmentation
brightness_augmentation = False
shift_augmentation = False

# Augmentation 1 (Rotate Image)

In [ ]:
def rotate_augmentation(images, keypoints): # Function to rotate image
    rotated_images = []
    rotated_keypoints = []
    print("Augmentasi dengan sudut sebesar (derajat): ")
    for angle in rotation_angles:
        for angle in [angle,-angle]:
            print(f'{angle}', end='  ')
            M = cv2.getRotationMatrix2D((48,48), angle, 1.0)
            angle_rad = -angle*pi/180.     # mengambil nilai sudut dalam radian
            # untuk nilai x
            for image in images:
                rotated_image = cv2.warpAffine(image, M, (96,96), flags=cv2.INTER_CUBIC)
                rotated_images.append(rotated_image)
            # untuk nilai y
            for keypoint in keypoints:
                keypoint = keypoint * 48 + 48 # Menyesuaikan nilai y untuk dirotasi
                rotated_keypoint = keypoint - 48
                for idx in range(0,len(rotated_keypoint),2):
                    rotated_keypoint[idx] = rotated_keypoint[idx]*cos(angle_rad)-rotated_keypoint[idx+1]*sin(angle_rad)
                    rotated_keypoint[idx+1] = rotated_keypoint[idx]*sin(angle_rad)+rotated_keypoint[idx+1]*cos(angle_rad)
                rotated_keypoint += 48
                rotated_keypoint = (rotated_keypoint - 48) / 48  # scale target coordinates to [-1, 1]

                rotated_keypoints.append(rotated_keypoint)

    return np.reshape(rotated_images,(-1,96,96,1)), rotated_keypoints

# Rotating images
rotated_train_images, rotated_train_keypoints = rotate_augmentation(x_train, y_train)
print("\nShape of rotated_train_images: {}".format(np.shape(rotated_train_images)))
print("Shape of rotated_train_keypoints: {}\n".format(np.shape(rotated_train_keypoints)))
rotated_train_images = rotated_train_images.reshape(rotated_train_images.shape[0],96,96,1)
plot_sample(rotated_train_images[10].reshape(96,96), rotated_train_keypoints[10])

if rotation_augmentation:
    x_train = np.concatenate((x_train_original, rotated_train_images))
    y_train = np.concatenate((y_train_original, rotated_train_keypoints))

# Augmentation 2 (Alter Image Brightness)

In [ ]:
def alter_brightness(images, keypoints): # Function to alter image brightness
    altered_brightness_images = []
    inc_brightness_images = np.clip(images*1.4, 0.0, 1.0)    # Gambar menjadi lebih terang
    dec_brightness_images = np.clip(images*0.6, 0.0, 1.0)    # Gambar menjadi lebih gelap
    altered_brightness_images.extend(inc_brightness_images)
    altered_brightness_images.extend(dec_brightness_images)
    return altered_brightness_images, np.concatenate((keypoints, keypoints))

# Altering images brightness
altered_brightness_train_images, altered_brightness_train_keypoints = alter_brightness(x_train_original, y_train_original)
print(f"Shape of altered_brightness_train_images: {np.shape(altered_brightness_train_images)}")
print(f"Shape of altered_brightness_train_keypoints: {np.shape(altered_brightness_train_keypoints)}")
altered_brightness_train_images = np.reshape(altered_brightness_train_images,(np.shape(altered_brightness_train_images)[0],96,96,1))
plot_sample(altered_brightness_train_images[10].reshape(96,96), altered_brightness_train_keypoints[10]) 

if brightness_augmentation:
    x_train = np.concatenate((x_train, altered_brightness_train_images))
    y_train = np.concatenate((y_train, altered_brightness_train_keypoints))

# Augmentation 3 (Shift Image)

In [ ]:
def shift_images(images, keypoints): # Function to shift image vertically and horizontally
    shifted_images = []
    shifted_keypoints = []
    final_keypoints = []
    for shift in pixel_shifts:
        for (shift_x,shift_y) in [(-shift,-shift),(-shift,shift),(shift,-shift),(shift,shift)]:
            M = np.float32([[1,0,shift_x],[0,1,shift_y]])
            for image, keypoint in zip(images, keypoints):
                shifted_image = cv2.warpAffine(image, M, (96,96), flags=cv2.INTER_CUBIC)
                keypoint = keypoint * 48 + 48 # Menyesuaikan nilai y untuk digeser
                shifted_keypoint = np.array([(point+shift_x) if idx%2==0 else (point+shift_y) for idx, point in enumerate(keypoint)])
                if np.all(0.0<shifted_keypoint) and np.all(shifted_keypoint<96.0):
                    shifted_images.append(shifted_image.reshape(96,96,1))
                    shifted_keypoints.append(shifted_keypoint)
    shifted_keypoints = np.clip(shifted_keypoints,0.0,96.0)

    for keypoint in shifted_keypoints:
      keypoint = (keypoint - 48) / 48
      final_keypoints.append(keypoint)

    return shifted_images, final_keypoints

# Shifting images
shifted_train_keypoints = []
shifted_train_images, shifted_train_keypoints = shift_images(x_train_original, y_train_original)
print(f"Shape of shifted_train_images: {np.shape(shifted_train_images)}")
print(f"Shape of shifted_train_keypoints: {np.shape(shifted_train_keypoints)}")
plot_sample(shifted_train_images[10].reshape(96,96), shifted_train_keypoints[10])

if shift_augmentation:
    x_train = np.concatenate((x_train, shifted_train_images))
    y_train = np.concatenate((y_train, shifted_train_keypoints))

In [ ]:
print(x_train.shape)

# Create Model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Predefined parameters
input_shape = (96, 96, 1)
output_shape = 30

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(64, kernel_size=(3, 3), activation="LeakyReLU", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(96, kernel_size=(3, 3), activation="LeakyReLU", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="LeakyReLU", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(256, kernel_size=(3, 3), activation="LeakyReLU", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(512, kernel_size=(3, 3), activation="LeakyReLU", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation="tanh"),
        layers.Dense(256, activation="ReLU"),
        layers.Dense(128, activation="ReLU"),
        layers.Dropout(0.1),
        layers.Dense(output_shape, activation="linear"),
    ]
)
model.summary()

# Train Model

In [ ]:
batch_size = 128
epochs = 100

from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(loss=root_mean_squared_error, optimizer="adam")
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Prediction and Submission

In [ ]:
pred = model.predict(x_test)
pred = pred * 48 # scale target coordinates to [0, 96]
pred += 48

In [ ]:
print(pred)

In [ ]:
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)

In [ ]:
rowid = lookid_data['RowId']
rowid=list(rowid)

In [ ]:
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))

In [ ]:
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])

In [ ]:
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')

In [ ]:
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('submission.csv',index = False)